# Assignment 4 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial 8 notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial 9 notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture 9 presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
!pip install wn
!python -m wn download omw-he:1.4

  Using cached https://files.pythonhosted.org/packages/ce/f1/53b07100f5c3d41fd33fc78ebb9e99d736b0460ced8acff94840311ffc60/wn-0.9.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/97/75/10a9ebee3fd790d20926a90a2547f0bf78f371b2f13aa822c759680ca7b9/tomli-2.0.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/75/e1/932e06004039dd670c9d5e1df0cd606bf46e29a28e65d5bb28e894ea29c9/typing_extensions-4.2.0-py3-none-any.whl



Cached file found: C:\Users\amity\.wn_data\downloads\7ecf10e89326bc0ac26ad94b40fe60a7b6ac3302

Checking C:\Users\amity\AppData\Local\Temp\tmpph_pd0kj\omw-he\omw-he.xml
Skipping omw-he:1.4 (Hebrew Wordnet); already added



In [4]:
# word net
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
!pip install hebrew_tokenizer

  Using cached https://files.pythonhosted.org/packages/87/35/a56b3f07eacd06cfded8ec18d9ebe3d6a4c63d31d056f076d860930b1a70/hebrew_tokenizer-2.3.0.tar.gz
  Stored in directory: C:\Users\amity\AppData\Local\pip\Cache\wheels\e9\6e\94\af608326c62b424b7d28edfa3190307b11de76f800a60f95aa
Successfully built hebrew-tokenizer


In [6]:
# tokenizer:
import hebrew_tokenizer as ht

C:\Users\amity\Desktop\למידת מכונה מטלה 4\עמית


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape
df_test.head()

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני..."
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...


In [9]:
df_test.head(3)

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


### Your implementation:
Write your code solution in the following code-cells

In [10]:
# text normal Function
def normalTxt(txt):
    txt = str(txt)
    txt = txt.lower()
    # Removing external punctuation marks
    txt = re.sub('\s\W',' ',txt)
    txt = re.sub('\W\s',' ',txt)
    #Arranging double spaces
    txt = re.sub('\s+',' ',txt)
    return txt

df_train['story'] = [normalTxt(txt) for txt in df_train['story']]
# Matrix of token 
vectorizer = CountVectorizer()
# Vocabulari dictionary 
x = vectorizer.fit_transform(df_train['story'])
# Replace the gender label to one encoding
encoderLabel = LabelEncoder()
y = encoderLabel.fit_transform(df_train['gender'])
# Split the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.35)



In [ ]:
# Make the neural network.
nbW=MLPClassifier(hidden_layer_sizes=(200,150,70), max_iter=300,activation = 'relu',solver='adam',random_state=2,shuffle=False)
# train the model
nbW.fit(x_train, y_train)

# calculate f1 and display
confusion_matrix(y_test,nbW.predict(x_test))
print("Macro f1 score:",f1_score(y_test, nbW.predict(x_test), average='macro'))
print('\n')
print(metrics.classification_report(y_test, nbW.predict(x_test)))
print(x_test.shape)
print(nbW.predict(x_test))
print(nbW.score(x_test, y_test)*100)

In [ ]:
# normal the test according to the train data
df_test['story'] = [normalTxt(txt) for txt in df_test['story']]
x = vectorizer.transform(df_test['story'])
print(x.shape)
# Predict the results on the test
test_prediction = nbW.predict(x)
for t in test_prediction:
    print(t)
# create data frame with the real label make or female
df_predicted = df_test[['test_example_id']].copy()
df_predicted['predicted_category'] = encoderLabel.inverse_transform(test_prediction)
print(df_predicted)


In [ ]:
# Add as many code cells as you need
# YOUR CODE HERE


### Save output to csv
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [ ]:
df_predicted.to_csv('classification_results.csv',index=False)